In [27]:
from active_learning.learner.standard import Learner, get_classifier
from active_learning.weak_supervision.SelfTraining import SelfTraining
import argparse
import numpy as np
import copy
import pandas as pd
import math
import random
from sklearn.metrics import accuracy_score, f1_score
from timeit import default_timer as timer
from typing import List, Dict
from active_learning.config import get_active_config
from active_learning.dataStorage import DataStorage
from active_learning.datasets import load_synthetic
from active_learning.logger import init_logger
from active_learning.merge_weak_supervision_label_strategies.MajorityVoteLabelMergeStrategy import (
    MajorityVoteLabelMergeStrategy,
)
from collections import Counter

from active_learning.weak_supervision import SyntheticLabelingFunctions
from active_learning.weak_supervision.BaseWeakSupervision import BaseWeakSupervision

import seaborn as sns
import matplotlib.pyplot as plt
#sns.set_theme(style="whitegrid")


font_size = 8

tex_fonts = {
    # Use LaTeX to write all text
    # "text.usetex": True,
    "text.usetex": False,
    "font.family": ["Times New Roman", 'sans-serif'],
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": font_size,
    "font.size": font_size,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": font_size,
    "xtick.labelsize": font_size,
    "ytick.labelsize": font_size,
    "xtick.bottom": True,
    "figure.autolayout": True,
}

sns.set_style("whitegrid")
sns.set_context("paper")
plt.rcParams.update(tex_fonts)  # type: ignore


# https://jwalton.info/Embed-Publication-Matplotlib-Latex/
def set_matplotlib_size(width, fraction=1):
    """Set figure dimensions to avoid scaling in LaTeX.

    Parameters
    ----------
    width: float
            Document textwidth or columnwidth in pts
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy

    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    # Width of figure (in pts)
    fig_width_pt = width * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    golden_ratio = (5 ** 0.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim


#width = 505.89
width = 1500


df = pd.read_csv("~/exp_results/run_how_to/exp_results.csv")
df

,n_samples,n_features,n_informative,n_redundant,n_repeated,n_classes,n_clusters_per_class,weights,flip_y,class_sep,...,JOB_ID,AL_SAMPLES_WEIGHT,AMOUNT_OF_LFS,CLUSTERED_AL_WS_COMBINATION,DATASET,DATASET_RANDOM_GENERATION_SEED,FRACTION_OF_INITIALLY_LABELLED_SAMPLES,FRACTION_OF_LASTLY_AL_LABELLED_SAMPLES,LF_RANDOM_SEED,MERGE_WS_SAMPLES_STRATEGY
0,178,14,14,0,0,3,?,?,?,?,...,21,62,1.667661,True,wine,914092,0.244126,0.168291,61088,MajorityVoteLabelMergeStrategy
1,100,21,21,0,0,2,?,?,?,?,...,19,62,3.462121,False,FERTILITY,408924,0.356298,0.906828,685441,MajorityVoteLabelMergeStrategy
2,351,35,35,0,0,2,?,?,?,?,...,20,27,2.673670,False,IONOSPHERE,579305,0.304781,0.164656,660891,MajorityVoteLabelMergeStrategy
3,100,21,21,0,0,2,?,?,?,?,...,4,47,4.153230,False,FERTILITY,328948,0.592415,0.046450,528179,RandomLabelMergeStrategy
4,306,4,4,0,0,2,?,?,?,?,...,2,38,1.001795,False,HABERMAN,214177,0.304242,0.524756,258796,MajorityVoteLabelMergeStrategy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8475,48842,105,105,0,0,4,?,?,?,?,...,9918,53,4.143662,True,adult,100406,0.406360,0.073848,245517,RandomLabelMergeStrategy
8476,10000,3073,3073,0,0,10,?,?,?,?,...,9610,47,7.759934,True,cifar10,82165,0.294491,0.644126,66889,SnorkelLabelMergeStrategy
8477,48842,105,105,0,0,4,?,?,?,?,...,9848,14,4.310431,True,adult,257669,0.310385,0.922679,212332,SnorkelLabelMergeStrategy
8478,10000,3073,3073,0,0,10,?,?,?,?,...,9732,66,7.084622,True,cifar10,242013,0.286538,0.003731,784560,RandomLabelMergeStrategy


In [28]:
df.keys()#

Index(['n_samples', 'n_features', 'n_informative', 'n_redundant', 'n_repeated',
       'n_classes', 'n_clusters_per_class', 'weights', 'flip_y', 'class_sep',
       'hypercube', 'scale', 'random_state', 'ABSTAIN_THRESHOLDS',
       'LF_CLASSIFIERS', 'AMOUNT_OF_LF_FEATURESSS', 'acc_WS', 'f1_WS',
       'f1_initial', 'acc_initial', 'f1_ws', 'acc_ws',
       'acc_ws_and_al_UncertaintyMaxMargin_no_ws',
       'f1_ws_and_al_UncertaintyMaxMargin_no_ws',
       'acc_al_and_al_UncertaintyMaxMargin_no_ws',
       'f1_al_and_al_UncertaintyMaxMargin_no_ws',
       'acc_ws_and_al_UncertaintyMaxMargin_with_ws',
       'f1_ws_and_al_UncertaintyMaxMargin_with_ws',
       'acc_al_and_al_UncertaintyMaxMargin_with_ws',
       'f1_al_and_al_UncertaintyMaxMargin_with_ws', 'acc_ws_and_al_Random',
       'f1_ws_and_al_Random', 'acc_al_and_al_Random', 'f1_al_and_al_Random',
       'acc_ws_and_al_CoveredByLeastAmountOfLf',
       'f1_ws_and_al_CoveredByLeastAmountOfLf',
       'acc_al_and_al_CoveredByLeastAmo

In [30]:
def hist_plot(df: pd.DataFrame, COMPARE_KEYS:List[str], TITLE:str=None, OUTPUT_PATH: str=None, LABEL:str="AUC-F1 (\\%)"):
    fig = plt.figure(figsize=set_matplotlib_size(width, fraction=0.5))

    # fig = plt.gcf()  # type: ignore
    # fig.set_size_inches(18.5, 10.5)

    selection_list = []
    for key in COMPARE_KEYS:
        selection_list.append((df[key], key))
    #for strategy in df["strategy"].unique():
    #    selection_list.append((df[df["strategy"] == strategy]["f1_auc"], strategy))

    for selection, label in sorted(selection_list, key=lambda tup: tup[1]):
        #  ax = sns.distplot(selection, label=label, **kwargs)
        #  label = "{}-{:>3}: {:.4g}% ".format(label[0], label[1], selection.mean() * 100)
        label = "{:<10}{:>5.4g}% ".format(label + ":", selection.mean() * 100)
        ax = sns.kdeplot(selection, label=label)
        # ax = sns.distplot(selection, label=label, kde=True)
        #ax.set_xlim(0, 1)
        ax.set_xlim(0.5, 0.6)
        #  ax.set_xlim(0.75, 0.8)
        #  ax.set_xlim(80, 875)

        mean = selection.mean()
        if selection.count() == 0:
            low = high = mean
        else:
            low = selection.mean() - 1.96 * selection.std() / math.sqrt(
                selection.count()
            )
            high = selection.mean() + 1.96 * selection.std() / math.sqrt(
                selection.count()
            )
        print(label)
        ax.axvline(mean, color=plt.gca().lines[-1].get_color())  # type: ignore
        ax.axvspan(low, high, alpha=0.2, color=plt.gca().lines[-1].get_color())  # type: ignore

        ax.set_xticks([x for x in ax.get_xticks()]) # stupid "workaround" for https://github.com/matplotlib/matplotlib/issues/18848
        ax.set_xticklabels(["{:.0%}\\%".format(x) for x in ax.get_xticks()])
    ax.set_title(TITLE)  # type: ignore
    # ax.set_ylabel("")
    ax.set_xlabel(LABEL)  # type: ignore
    ax.legend(  # type: ignore
        loc="center left"
    )  # , bbox_to_anchor=(0.0, 1.01), borderaxespad=0, frameon=False, ncol=len(selection))  # type: ignore
    # ax.xa

def comparison_table(df: pd.DataFrame, COMPARE_KEYS:List[str], TITLE:str=""):
    results = []
    for key in COMPARE_KEYS:
        selection = df[key]

        mean = selection.mean()
        error_area = 1.96 * selection.std() / math.sqrt(
                selection.count()
            )
        results.append([key, mean, error_area])
    print(TITLE)
    print("{:>60} {:>6} {:>4}".format("", "Mean", ""))
    for result in results:
        print("{:<50} {:>6.2%} ±{:>4.2%}".format(result[0], float(result[1]), float(result[2])))

def answer_questions(df: pd.DataFrame, COMPARE_KEYS:List[str], metric):
    results = {}
    for key in COMPARE_KEYS:
        selection = df[key]

        mean = selection.mean()
        error_area = 1.96 * selection.std() / math.sqrt(
                selection.count()
            )
        results[key] = (mean, error_area)
    
    better_than_initial = [k for k,v in results.items() if v[0] > results[metric + "_initial"][0]]
    better_than_initial_with_ws = [k for k in better_than_initial if "ws_and_al" in k]
    print("Which are better than initial?", better_than_initial)
    print("Which are better than initial, with WS?", better_than_initial_with_ws)

    al_ws_al_better_than_al_al = []

    for k in results.keys():
        if k.startswith(metric + "_ws_and_al"):
            if results[k][0] > results[k.replace("_ws_and_al", "_al_and_al")][0]:
                al_ws_al_better_than_al_al.append(k)

    print("Where is al+ws+al better than al+al?", al_ws_al_better_than_al_al)
    
    #print("Where is the mean improvement within the 95% error range?")
    
    
# 01 general improvement
for metric in ['acc', 'f1']:
    COMPARE_KEYS = [metric+"_initial", metric +"_ws"]
    for AL_SAMPLING_STRATEGY in ["UncertaintyMaxMargin_no_ws",
            "UncertaintyMaxMargin_with_ws",
            "Random",
            "CoveredByLeastAmountOfLf",
            "ClassificationIsMostWrong",
            "GreatestDisagreement"]:
            COMPARE_KEYS.append(metric + '_ws_and_al_'+AL_SAMPLING_STRATEGY)
            COMPARE_KEYS.append(metric + '_al_and_al_'+AL_SAMPLING_STRATEGY)
    comparison_table(df, COMPARE_KEYS=COMPARE_KEYS, TITLE="General Improvement after AL + WS " + metric)
    answer_questions(df, COMPARE_KEYS=COMPARE_KEYS, metric=metric)

General Improvement after AL + WS acc
                                                               Mean     
acc_initial                                        61.69% ±0.53%
acc_ws                                             61.80% ±0.52%
acc_ws_and_al_UncertaintyMaxMargin_no_ws           61.91% ±0.54%
acc_al_and_al_UncertaintyMaxMargin_no_ws           62.12% ±0.55%
acc_ws_and_al_UncertaintyMaxMargin_with_ws         61.91% ±0.54%
acc_al_and_al_UncertaintyMaxMargin_with_ws         62.12% ±0.55%
acc_ws_and_al_Random                               61.84% ±0.53%
acc_al_and_al_Random                               61.94% ±0.55%
acc_ws_and_al_CoveredByLeastAmountOfLf             61.80% ±0.53%
acc_al_and_al_CoveredByLeastAmountOfLf             62.02% ±0.54%
acc_ws_and_al_ClassificationIsMostWrong            61.79% ±0.52%
acc_al_and_al_ClassificationIsMostWrong            62.02% ±0.53%
acc_ws_and_al_GreatestDisagreement                 61.68% ±0.53%
acc_al_and_al_GreatestDisagreement          

In [31]:


# group by stuff
for group_by in ['DATASET', 'MERGE_WS_SAMPLES_STRATEGY']:
    print()
    print("#"*80)
    print(group_by)
    print("#"*80)
    for group_by_value in df[group_by].unique():
        print()
        print(group_by_value, ": ", len(df[df[group_by] == group_by_value]))
        for metric in ['acc', 'f1']:
            COMPARE_KEYS = [metric+"_initial", metric +"_ws"]
            for AL_SAMPLING_STRATEGY in ["UncertaintyMaxMargin_no_ws",
                    "UncertaintyMaxMargin_with_ws",
                    "Random",
                    "CoveredByLeastAmountOfLf",
                    "ClassificationIsMostWrong",
                    "GreatestDisagreement"]:
                    COMPARE_KEYS.append(metric + '_ws_and_al_'+AL_SAMPLING_STRATEGY)
                    COMPARE_KEYS.append(metric + '_al_and_al_'+AL_SAMPLING_STRATEGY)
            #comparison_table(df, COMPARE_KEYS=COMPARE_KEYS, TITLE="General Improvement after AL + WS " + metric)
            answer_questions(df[df[group_by] == group_by_value], COMPARE_KEYS=COMPARE_KEYS, metric=metric)


################################################################################
DATASET
################################################################################

wine :  271
Which are better than initial? ['acc_ws', 'acc_ws_and_al_UncertaintyMaxMargin_no_ws', 'acc_al_and_al_UncertaintyMaxMargin_no_ws', 'acc_ws_and_al_UncertaintyMaxMargin_with_ws', 'acc_al_and_al_UncertaintyMaxMargin_with_ws', 'acc_ws_and_al_Random', 'acc_al_and_al_Random', 'acc_ws_and_al_CoveredByLeastAmountOfLf', 'acc_al_and_al_CoveredByLeastAmountOfLf', 'acc_ws_and_al_ClassificationIsMostWrong', 'acc_al_and_al_ClassificationIsMostWrong', 'acc_ws_and_al_GreatestDisagreement', 'acc_al_and_al_GreatestDisagreement']
Which are better than initial, with WS? ['acc_ws_and_al_UncertaintyMaxMargin_no_ws', 'acc_ws_and_al_UncertaintyMaxMargin_with_ws', 'acc_ws_and_al_Random', 'acc_ws_and_al_CoveredByLeastAmountOfLf', 'acc_ws_and_al_ClassificationIsMostWrong', 'acc_ws_and_al_GreatestDisagreement']
Where is al+ws+al bett

In [34]:
#andersherum an die Sache gehen : im großen df schauen wo al_ws_al better than al+al -> und dann schauen wie viel prozent davon jeweils auf die einzelnen werte von datasets, … verteilt sind!!
df1 = pd.DataFrame({'city':['A','B','C'],
                    'keep1':['D', 'E', 'F'],
                    'f1_ws_and_al_U':[1,2,3],
                    'f1_al_and_al_U':[4,5,6],
                    'f1_ws_and_al_R':[7,8,9],
                    'f1_al_and_al_R':[10,11,12],
                    'f1_ws_and_al_X':[13,14,15],
                    'f1_al_and_al_X':[16,17,18],

                    'acc_ws_and_al_U':[19,20,21],
                    'acc_al_and_al_U':[22,23,24],
                    'acc_ws_and_al_R':[25,26,27],
                    'acc_al_and_al_R':[27,28,29],
                    'acc_ws_and_al_X':[30,31,32],
                    'acc_al_and_al_X':[33,34,35],
})
print(df1)




df_new = pd.DataFrame()
al_sampling_strategies = ["U", "R", "X"]
to_split_metrics = (
    ["f1_ws_and_al_" + k for k in al_sampling_strategies]
    + ["f1_al_and_al_" + k for k in al_sampling_strategies]
    + ["acc_ws_and_al_" + k for k in al_sampling_strategies]
    + ["acc_al_and_al_" + k for k in al_sampling_strategies]
)
print(to_split_metrics)

for _, row in df1.iterrows():
    for al_sampling_strategy in al_sampling_strategies:
        # remove all other metrics
        row2 = row.drop(
            [m for m in to_split_metrics if not m.endswith(al_sampling_strategy)]
        )
        row2 = row2.rename(
            {
                "f1_ws_and_al_" + al_sampling_strategy: "f1_al_and_ws_and_al",
                "acc_ws_and_al_" + al_sampling_strategy: "acc_al_and_ws_and_al",
                "f1_al_and_al_" + al_sampling_strategy: "f1_al_and_al",
                "acc_al_and_al_" + al_sampling_strategy: "acc_al_and_al",
            }
        )
        df_new = df_new.append(row2)

df_new.keys()

  city keep1  f1_ws_and_al_U  f1_al_and_al_U  f1_ws_and_al_R  f1_al_and_al_R  \
0    A     D               1               4               7              10   
1    B     E               2               5               8              11   
2    C     F               3               6               9              12   

   f1_ws_and_al_X  f1_al_and_al_X  acc_ws_and_al_U  acc_al_and_al_U  \
0              13              16               19               22   
1              14              17               20               23   
2              15              18               21               24   

   acc_ws_and_al_R  acc_al_and_al_R  acc_ws_and_al_X  acc_al_and_al_X  
0               25               27               30               33  
1               26               28               31               34  
2               27               29               32               35  
['f1_ws_and_al_U', 'f1_ws_and_al_R', 'f1_ws_and_al_X', 'f1_al_and_al_U', 'f1_al_and_al_R', 'f1_al_and_al_X

Index(['city', 'keep1', 'f1_al_and_ws_and_al', 'f1_al_and_al',
       'acc_al_and_ws_and_al', 'acc_al_and_al'],
      dtype='object')

In [35]:
CREATE_CSV = True
if CREATE_CSV:
    # unravel al sampling strat metrics
    al_sampling_strategies = [
        "UncertaintyMaxMargin_no_ws",
        "UncertaintyMaxMargin_with_ws",
        "Random",
        "CoveredByLeastAmountOfLf",
        "ClassificationIsMostWrong",
        "GreatestDisagreement",
    ]

    df_new = pd.DataFrame()
    to_split_metrics = (
        ["f1_ws_and_al_" + k for k in al_sampling_strategies]
        + ["f1_al_and_al_" + k for k in al_sampling_strategies]
        + ["acc_ws_and_al_" + k for k in al_sampling_strategies]
        + ["acc_al_and_al_" + k for k in al_sampling_strategies]
    )
    print(to_split_metrics)

    for _, row in df.iterrows():
        for al_sampling_strategy in al_sampling_strategies:
            # remove all other metrics
            row2 = row.drop(
                [m for m in to_split_metrics if not m.endswith(al_sampling_strategy)]
            )
            row2 = row2.rename(
                {
                    "f1_ws_and_al_" + al_sampling_strategy: "f1_al_and_ws_and_al",
                    "acc_ws_and_al_" + al_sampling_strategy: "acc_al_and_ws_and_al",
                    "f1_al_and_al_" + al_sampling_strategy: "f1_al_and_al",
                    "acc_al_and_al_" + al_sampling_strategy: "acc_al_and_al",
                }
            )
            row2['al_sampling_strategy'] = al_sampling_strategy
            df_new = df_new.append(row2)

    df_new
df_new.keys()

['f1_ws_and_al_UncertaintyMaxMargin_no_ws', 'f1_ws_and_al_UncertaintyMaxMargin_with_ws', 'f1_ws_and_al_Random', 'f1_ws_and_al_CoveredByLeastAmountOfLf', 'f1_ws_and_al_ClassificationIsMostWrong', 'f1_ws_and_al_GreatestDisagreement', 'f1_al_and_al_UncertaintyMaxMargin_no_ws', 'f1_al_and_al_UncertaintyMaxMargin_with_ws', 'f1_al_and_al_Random', 'f1_al_and_al_CoveredByLeastAmountOfLf', 'f1_al_and_al_ClassificationIsMostWrong', 'f1_al_and_al_GreatestDisagreement', 'acc_ws_and_al_UncertaintyMaxMargin_no_ws', 'acc_ws_and_al_UncertaintyMaxMargin_with_ws', 'acc_ws_and_al_Random', 'acc_ws_and_al_CoveredByLeastAmountOfLf', 'acc_ws_and_al_ClassificationIsMostWrong', 'acc_ws_and_al_GreatestDisagreement', 'acc_al_and_al_UncertaintyMaxMargin_no_ws', 'acc_al_and_al_UncertaintyMaxMargin_with_ws', 'acc_al_and_al_Random', 'acc_al_and_al_CoveredByLeastAmountOfLf', 'acc_al_and_al_ClassificationIsMostWrong', 'acc_al_and_al_GreatestDisagreement']


Index(['n_samples', 'n_features', 'n_informative', 'n_redundant', 'n_repeated',
       'n_classes', 'n_clusters_per_class', 'weights', 'flip_y', 'class_sep',
       'hypercube', 'scale', 'random_state', 'ABSTAIN_THRESHOLDS',
       'LF_CLASSIFIERS', 'AMOUNT_OF_LF_FEATURESSS', 'acc_WS', 'f1_WS',
       'f1_initial', 'acc_initial', 'f1_ws', 'acc_ws', 'acc_al_and_ws_and_al',
       'f1_al_and_ws_and_al', 'acc_al_and_al', 'f1_al_and_al',
       'amount_of_initial_al_samples', 'amount_of_lastly_al_samples',
       'amount_of_ws_labelled_samples', 'JOB_ID', 'AL_SAMPLES_WEIGHT',
       'AMOUNT_OF_LFS', 'CLUSTERED_AL_WS_COMBINATION', 'DATASET',
       'DATASET_RANDOM_GENERATION_SEED',
       'FRACTION_OF_INITIALLY_LABELLED_SAMPLES',
       'FRACTION_OF_LASTLY_AL_LABELLED_SAMPLES', 'LF_RANDOM_SEED',
       'MERGE_WS_SAMPLES_STRATEGY', 'al_sampling_strategy'],
      dtype='object')

In [40]:
if CREATE_CSV:
    df_new.to_csv("how_to.csv", index=False)

In [39]:
df = pd.read_csv("how_to.csv")

print(df.keys())
print("Total: ", len(df))

no_hist_keys = [
    "ABSTAIN_THRESHOLDS",
    "AL_SAMPLES_WEIGTHS",
    "DATASET_RANDOM_GENERATION_SEED",
    "JOB_ID",
    "LF_CLASSIFIERS",
    "LF_RANDOM_SEED",
    "random_state",
    "AMOUNT_OF_LF_FEATURESSS",
]

hist_keys_that_are_lists = [
    "ABSTAIN_THRESHOLDS",
    "LF_CLASSIFIERS",
    "AMOUNT_OF_LF_FEATURESSS",
]

# 1. check the value ranges -> does it make sense to go one_hot_encoding? for LF_CLASSIFIERS definitely
# 2. for the rest not so much -,-


title="all"
'''
for key in df:
    if key.startswith("acc_") or key.startswith("f1_") or key in no_hist_keys:
        continue
    print(key)
    sns.histplot(df, y=key).set(title=title)
    plt.savefig('plots/' + title+ '_' + key + '.jpg', dpi=300, bbox_inches="tight")
    plt.savefig('plots/' + title+ '_' + key + '.pdf',dpi=300, format="pdf", bbox_inches="tight")
    plt.clf()


def analyse_value_distributions(df, df_a, df_not_a, title):
    for key in df_a:
        if key.startswith("acc_") or key.startswith("f1_") or key in no_hist_keys:
            continue
        print(key)

        f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, sharex=True)
        ax1.set_title(title)
        sns.histplot(df_a, y=key, ax=ax1)
        sns.histplot(df_not_a, y=key, ax=ax2)
        plt.savefig("plots/" + title + "_" + key + ".jpg", dpi=300, bbox_inches="tight")
        plt.savefig(
            "plots/" + title + "_" + key + ".pdf",
            dpi=300,
            format="pdf",
            bbox_inches="tight",
        )
        plt.clf()
        #acc per lf als stärke der lf

for metric in ["acc", "f1"]:
    print(metric)
        
    analyse_value_distributions(
        df,
        df.loc[(df[metric + "_initial"] > df[metric + "_al_and_al"])
            & (df[metric + "_initial"] > df[metric + "_al_and_ws_and_al"])],
        df.loc[(df[metric + "_initial"] <= df[metric + "_al_and_al"])
            & (df[metric + "_initial"] <= df[metric + "_al_and_ws_and_al"])],
        metric + " Initial was better than everything else",
    )
    analyse_value_distributions(
        df,
        df.loc[(df[metric + "_ws"] > df[metric + "_initial"])],
        df.loc[(df[metric + "_ws"] >= df[metric + "_initial"])],
         metric + "Only WS > Initial",
    )
    analyse_value_distributions(
        df,
        df.loc[(df[metric + "_al_and_ws_and_al"] > df[metric + "_ws"])],
        df.loc[(df[metric + "_al_and_ws_and_al"] <= df[metric + "_ws"])],
         metric + " AL and WS and AL > Only WS",
    )
    analyse_value_distributions(
        df,
        df.loc[ (df[metric + "_al_and_al"] > df[metric + "_initial"])],
        df.loc[ (df[metric + "_al_and_al"] <= df[metric + "_initial"])],
         metric + " AL and AL > AL",
    )
    analyse_value_distributions(
        df,
        df.loc[(df[metric + "_al_and_ws_and_al"] < df[metric + "_ws"])],
        df.loc[(df[metric + "_al_and_ws_and_al"] >= df[metric + "_ws"])],
        metric + " AL and WS and AL < Only WS",
    )'''
df

Index(['n_samples', 'n_features', 'n_informative', 'n_redundant', 'n_repeated',
       'n_classes', 'n_clusters_per_class', 'weights', 'flip_y', 'class_sep',
       'hypercube', 'scale', 'random_state', 'ABSTAIN_THRESHOLDS',
       'LF_CLASSIFIERS', 'AMOUNT_OF_LF_FEATURESSS', 'acc_WS', 'f1_WS',
       'f1_initial', 'acc_initial', 'f1_ws', 'acc_ws', 'acc_al_and_ws_and_al',
       'f1_al_and_ws_and_al', 'acc_al_and_al', 'f1_al_and_al',
       'amount_of_initial_al_samples', 'amount_of_lastly_al_samples',
       'amount_of_ws_labelled_samples', 'JOB_ID', 'AL_SAMPLES_WEIGHT',
       'AMOUNT_OF_LFS', 'CLUSTERED_AL_WS_COMBINATION', 'DATASET',
       'DATASET_RANDOM_GENERATION_SEED',
       'FRACTION_OF_INITIALLY_LABELLED_SAMPLES',
       'FRACTION_OF_LASTLY_AL_LABELLED_SAMPLES', 'LF_RANDOM_SEED',
       'MERGE_WS_SAMPLES_STRATEGY', 'al_sampling_strategy'],
      dtype='object')
Total:  50880


/home/julius/.local/share/virtualenvs/code-oYwF_TsS/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,n_samples,n_features,n_informative,n_redundant,n_repeated,n_classes,n_clusters_per_class,weights,flip_y,class_sep,...,AL_SAMPLES_WEIGHT,AMOUNT_OF_LFS,CLUSTERED_AL_WS_COMBINATION,DATASET,DATASET_RANDOM_GENERATION_SEED,FRACTION_OF_INITIALLY_LABELLED_SAMPLES,FRACTION_OF_LASTLY_AL_LABELLED_SAMPLES,LF_RANDOM_SEED,MERGE_WS_SAMPLES_STRATEGY,al_sampling_strategy
0,178.0,14.0,14.0,0.0,0.0,3.0,?,?,?,?,...,62.0,1.667661,1.0,wine,914092.0,0.244126,0.168291,61088.0,MajorityVoteLabelMergeStrategy,UncertaintyMaxMargin_no_ws
1,178.0,14.0,14.0,0.0,0.0,3.0,?,?,?,?,...,62.0,1.667661,True,wine,914092.0,0.244126,0.168291,61088.0,MajorityVoteLabelMergeStrategy,UncertaintyMaxMargin_with_ws
2,178.0,14.0,14.0,0.0,0.0,3.0,?,?,?,?,...,62.0,1.667661,True,wine,914092.0,0.244126,0.168291,61088.0,MajorityVoteLabelMergeStrategy,Random
3,178.0,14.0,14.0,0.0,0.0,3.0,?,?,?,?,...,62.0,1.667661,True,wine,914092.0,0.244126,0.168291,61088.0,MajorityVoteLabelMergeStrategy,CoveredByLeastAmountOfLf
4,178.0,14.0,14.0,0.0,0.0,3.0,?,?,?,?,...,62.0,1.667661,True,wine,914092.0,0.244126,0.168291,61088.0,MajorityVoteLabelMergeStrategy,ClassificationIsMostWrong
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50875,10000.0,3073.0,3073.0,0.0,0.0,10.0,?,?,?,?,...,33.0,7.216152,True,cifar10,484338.0,0.094111,0.399906,501539.0,MajorityVoteLabelMergeStrategy,UncertaintyMaxMargin_with_ws
50876,10000.0,3073.0,3073.0,0.0,0.0,10.0,?,?,?,?,...,33.0,7.216152,True,cifar10,484338.0,0.094111,0.399906,501539.0,MajorityVoteLabelMergeStrategy,Random
50877,10000.0,3073.0,3073.0,0.0,0.0,10.0,?,?,?,?,...,33.0,7.216152,True,cifar10,484338.0,0.094111,0.399906,501539.0,MajorityVoteLabelMergeStrategy,CoveredByLeastAmountOfLf
50878,10000.0,3073.0,3073.0,0.0,0.0,10.0,?,?,?,?,...,33.0,7.216152,True,cifar10,484338.0,0.094111,0.399906,501539.0,MajorityVoteLabelMergeStrategy,ClassificationIsMostWrong


In [41]:
import ast
from collections import defaultdict

df = pd.read_csv("how_to.csv")

hist_keys_that_are_lists =  [
    "ABSTAIN_THRESHOLDS", # first binarize, then count values?
    "LF_CLASSIFIERS", # -> #dt #knn #lr -> count values
    "AMOUNT_OF_LF_FEATURESSS", # -> #1lfs #2lfs #3fs -> count values
]
key_values = {}
for key in hist_keys_that_are_lists:
    key_values[key] = defaultdict(lambda: 0)

    for row in df[key]:
        if key == 'AL_SAMPLES_WEIGHT':
            parsed_object = int(row)
        else:
            parsed_object = ast.literal_eval(row)

        if isinstance(parsed_object, list):
            for value in parsed_object:
                if key != 'ABSTAIN_THRESHOLDS':
                    key_values[key][value] += 1 
                else:
                    key_values[key][round(value,1)] += 1
        else:
            key_values[key][10*round(parsed_object/10)] += 1
    #print(key)
    #print(key_values)
print(key_values)
CREATE_CSV = True
if CREATE_CSV:
    # initialize new colums with 0
    for k,v in key_values.items():
        for value in v.keys():
            df[k+'_'+str(value)] = 0

    for key in hist_keys_that_are_lists:
        print(key)
        for ix, row in df[key].iteritems():

            if key == 'AL_SAMPLES_WEIGHT':
                parsed_object = int(row)
            else:
                parsed_object = ast.literal_eval(row)
                
            if isinstance(parsed_object, list):
                for value in parsed_object:
                    if key != 'ABSTAIN_THRESHOLDS':
                        df.loc[ix, key + '_' + str(value)] += 1
                    else:
                        df.loc[ix, key + '_' + str(round(value,1))] += 1
            else:
                df.loc[ix, key + '_' + str(10*round(parsed_object/10))] += 1
    print(df.keys())
    df.drop(hist_keys_that_are_lists, axis=1, inplace=True)
    print(df.keys())
    df

/home/julius/.local/share/virtualenvs/code-oYwF_TsS/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


{'ABSTAIN_THRESHOLDS': defaultdict(<function <lambda> at 0x7f1070d290d0>, {0.2: 23916, 0.8: 23682, 0.4: 24228, 0.7: 23604, 0.1: 24060, 0.5: 23694, 0.0: 12318, 0.6: 24570, 0.3: 24162, 0.9: 23868, 1.0: 11640}), 'LF_CLASSIFIERS': defaultdict(<function <lambda> at 0x7f106fe34280>, {'dt': 80352, 'lr': 79866, 'knn': 79524}), 'AMOUNT_OF_LF_FEATURESSS': defaultdict(<function <lambda> at 0x7f106fe3c820>, {1: 145410, 3: 24174, 2: 41256, 4: 16524, 5: 12378})}
ABSTAIN_THRESHOLDS
LF_CLASSIFIERS
AMOUNT_OF_LF_FEATURESSS
Index(['n_samples', 'n_features', 'n_informative', 'n_redundant', 'n_repeated',
       'n_classes', 'n_clusters_per_class', 'weights', 'flip_y', 'class_sep',
       'hypercube', 'scale', 'random_state', 'ABSTAIN_THRESHOLDS',
       'LF_CLASSIFIERS', 'AMOUNT_OF_LF_FEATURESSS', 'acc_WS', 'f1_WS',
       'f1_initial', 'acc_initial', 'f1_ws', 'acc_ws', 'acc_al_and_ws_and_al',
       'f1_al_and_ws_and_al', 'acc_al_and_al', 'f1_al_and_al',
       'amount_of_initial_al_samples', 'amount_of_l

In [42]:
df.to_csv('new.csv', index=False)
df.keys()

Index(['n_samples', 'n_features', 'n_informative', 'n_redundant', 'n_repeated',
       'n_classes', 'n_clusters_per_class', 'weights', 'flip_y', 'class_sep',
       'hypercube', 'scale', 'random_state', 'acc_WS', 'f1_WS', 'f1_initial',
       'acc_initial', 'f1_ws', 'acc_ws', 'acc_al_and_ws_and_al',
       'f1_al_and_ws_and_al', 'acc_al_and_al', 'f1_al_and_al',
       'amount_of_initial_al_samples', 'amount_of_lastly_al_samples',
       'amount_of_ws_labelled_samples', 'JOB_ID', 'AL_SAMPLES_WEIGHT',
       'AMOUNT_OF_LFS', 'CLUSTERED_AL_WS_COMBINATION', 'DATASET',
       'DATASET_RANDOM_GENERATION_SEED',
       'FRACTION_OF_INITIALLY_LABELLED_SAMPLES',
       'FRACTION_OF_LASTLY_AL_LABELLED_SAMPLES', 'LF_RANDOM_SEED',
       'MERGE_WS_SAMPLES_STRATEGY', 'al_sampling_strategy',
       'ABSTAIN_THRESHOLDS_0.2', 'ABSTAIN_THRESHOLDS_0.8',
       'ABSTAIN_THRESHOLDS_0.4', 'ABSTAIN_THRESHOLDS_0.7',
       'ABSTAIN_THRESHOLDS_0.1', 'ABSTAIN_THRESHOLDS_0.5',
       'ABSTAIN_THRESHOLDS_0.0', 'A